# Notes:

---

Use this to explore database and query smaller tables for later analysis.

Save queries intended for further analysis in 'data/' so you do not have to run the query every time.

We should be able to query subsets of this data that will be small enough to store in RAM


In [1]:
import os
import duckdb
import pandas as pd
import numpy as np
import re
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
#from sentence_transformers import SentenceTransformer
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.utils import to_dense_adj
import matplotlib.pyplot as plt

# os.getcwd()

## CONNECT TO DB ##
# access data on disk by explicitly passing db file
# db and query results will be stored on-disk instead of RAM
# storing results as pandas df will use RAM normally
con = duckdb.connect("../data/database.sqlite")


## QUERY FUNCTION ##
# simple query function for initital exploration
# feel free to add functionality to this
def simple_query(query, params=None):
    # optional params for easy sanitized inputs
    # use ? in query to pass variables safely into query
    # pass params as list of variable names to be passed to query
    # results are pandas df
    result = con.execute(query, params or []).fetchdf()
    return result


# Query Data

In [3]:
query = """
select * from May2015
where subreddit = 'politics'
"""
politics = simple_query(query)

RuntimeError: Query interrupted

In [151]:
print(politics.shape)
politics.memory_usage(deep=True).sum()/(1024 ** 2)

(244927, 22)


np.float64(308.6400547027588)

In [135]:
def decode(x):
    if isinstance(x, (bytes, bytearray)):
        return x.decode("utf-8", errors="ignore")
    else:
        return x
politics_df = politics.copy(deep=True)

for col in politics_df.columns:
    politics_df[col] = politics_df[col].apply(decode)

politics_df["created_utc"] = pd.to_datetime(politics_df["created_utc"], unit="s", utc=True)


In [138]:
# saves movies dataframe to parquet
politics_df.to_parquet('../data/politics_subreddit.parquet', engine='pyarrow', index=False, compression='snappy')
politics_df.head()


,created_utc,ups,subreddit_id,link_id,name,score_hidden,author_flair_css_class,author_flair_text,subreddit,id,...,downs,archived,author,score,retrieved_on,body,distinguished,edited,controversiality,parent_id
0,2015-05-01 00:00:00+00:00,0,t5_2cneq,t3_34f7mc,t1_cqug90i,0,<NA>,<NA>,politics,cqug90i,...,0,0,Wicked_Truth,0,1432703079,Are you really implying we return to those tim...,<NA>,0,0,t1_cqufim0
1,2015-05-01 00:00:06+00:00,-1,t5_2cneq,t3_34fn91,t1_cqug95d,0,<NA>,<NA>,politics,cqug95d,...,0,0,OceanGroovedropper,-1,1432703081,"Seems morally questionable to me, but if that'...",<NA>,0,0,t1_cqufu3n
2,2015-05-01 00:00:24+00:00,1,t5_2cneq,t3_34ebvj,t1_cqug9j2,0,<NA>,<NA>,politics,cqug9j2,...,0,0,Drooperdoo,1,1432703086,"Ah, Ninjew, we feel the same.\n\nWait! Nin*jew...",<NA>,0,0,t1_cqug1n2
3,2015-05-01 00:00:33+00:00,1,t5_2cneq,t3_34a6iq,t1_cqug9p6,0,<NA>,<NA>,politics,cqug9p6,...,0,0,PabloNueve,1,1432703088,I mean I suppose we'll find out. I just have n...,<NA>,0,0,t1_cqtsyye
4,2015-05-01 00:00:50+00:00,2,t5_2cneq,t3_34e3wn,t1_cquga2u,0,<NA>,<NA>,politics,cquga2u,...,0,0,whosename,2,1432703093,Nor should you be able to. \nWould you like...,<NA>,0,0,t1_cqu59v3


# Analysis

In [2]:
politics_df = pd.read_parquet('../data/politics_subreddit.parquet', engine="pyarrow")
politics_df.head()

,created_utc,ups,subreddit_id,link_id,name,score_hidden,author_flair_css_class,author_flair_text,subreddit,id,...,downs,archived,author,score,retrieved_on,body,distinguished,edited,controversiality,parent_id
0,2015-05-01 00:00:00+00:00,0,t5_2cneq,t3_34f7mc,t1_cqug90i,0,None,None,politics,cqug90i,...,0,0,Wicked_Truth,0,1432703079,Are you really implying we return to those tim...,None,0,0,t1_cqufim0
1,2015-05-01 00:00:06+00:00,-1,t5_2cneq,t3_34fn91,t1_cqug95d,0,None,None,politics,cqug95d,...,0,0,OceanGroovedropper,-1,1432703081,"Seems morally questionable to me, but if that'...",None,0,0,t1_cqufu3n
2,2015-05-01 00:00:24+00:00,1,t5_2cneq,t3_34ebvj,t1_cqug9j2,0,None,None,politics,cqug9j2,...,0,0,Drooperdoo,1,1432703086,"Ah, Ninjew, we feel the same.\n\nWait! Nin*jew...",None,0,0,t1_cqug1n2
3,2015-05-01 00:00:33+00:00,1,t5_2cneq,t3_34a6iq,t1_cqug9p6,0,None,None,politics,cqug9p6,...,0,0,PabloNueve,1,1432703088,I mean I suppose we'll find out. I just have n...,None,0,0,t1_cqtsyye
4,2015-05-01 00:00:50+00:00,2,t5_2cneq,t3_34e3wn,t1_cquga2u,0,None,None,politics,cquga2u,...,0,0,whosename,2,1432703093,Nor should you be able to. \nWould you like...,None,0,0,t1_cqu59v3


In [3]:
cols = ["created_utc", "ups", "downs", "score", "subreddit", "author", "body", "parent_id", "link_id",'id']
df = politics_df[cols].sort_values("created_utc")

In [4]:


### Cleaning
df = df[df["author"].notnull()]
df = df[~df["author"].isin(["[deleted]", "AutoModerator"])]

### Adding features
df["body_len"] = df["body"].str.len().fillna(0)
df["time_diff"] = df.groupby("author")["created_utc"].diff().dt.total_seconds()

features = df.groupby("author").agg({
    "id": "count",        # total comments
    "ups": "mean",         #upvotes
    "downs": "mean",        #downvotes
    "score": "mean",
    "body_len": "mean",     #avg post length
    "time_diff": "median"  # posting frequency
}).reset_index()

features.columns = ["author", "n_comments", "avg_ups", "avg_downs", "avg_score", "avg_body_len", "median_time_diff"]



# text features
def lexical_diversity(texts):
    tokens = re.findall(r'\b\w+\b', ' '.join(texts).lower())
    if not tokens:
        return 0
    return len(set(tokens)) / len(tokens)

word_diversity = (
    df.groupby('author')['body']
    .apply(lexical_diversity)
    .reset_index(name='lexical_diversity')
)


# temporal features
df['hour'] = df['created_utc'].dt.hour
hour_counts = df.groupby(['author', 'hour']).size().unstack(fill_value=0)
hour_fraction = hour_counts.div(hour_counts.sum(axis=1), axis=0)

def entropy(p):
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

hour_features = pd.DataFrame({
    'author': hour_fraction.index,
    'active_hours': (hour_fraction > 0).sum(axis=1),
    'activity_entropy': hour_fraction.apply(entropy, axis=1)
})


# merge

word_diversity = word_diversity.reset_index(drop=True)
hour_features = hour_features.reset_index(drop=True)
features = features.reset_index(drop=True)

user_features = (
    features
    .merge(word_diversity, on="author", how="outer")
    .merge(hour_features, on="author", how="outer")
    .fillna(0)
)


# language modeling
'''
text_model = SentenceTransformer('all-MiniLM-L6-v2')
def avg_similarity(texts):
    if len(texts) < 2:
        return np.nan
    embeddings = text_model.encode(texts, show_progress_bar=False)
    sims = np.inner(embeddings, embeddings)
    upper = np.triu_indices_from(sims, k=1)
    return sims[upper].mean()

text_similarity = (
    df.groupby('author')['body']
    .apply(lambda x: avg_similarity(x.sample(min(len(x), 50), random_state=42)))  # limit to 50 per author
    .reset_index(name='avg_text_similarity')
)
'''
df.shape

(229108, 13)

In [5]:
# graph creation

# Extract author and parent info
comments = df[['author', 'parent_id', 'id']].copy()

# remove t1/2/3_ from id
comments['parent_id_clean'] = comments['parent_id'].str.replace(r'^t\d_', '', regex=True)
comments['id_clean'] = comments['id'].str.replace(r'^t\d_', '', regex=True)

#
merged = comments.merge(
    comments[['id_clean', 'author']],
    left_on='parent_id_clean',
    right_on='id_clean',
    how='left',
    suffixes=('', '_parent')
)

# drop duplicates and self connections
edges = merged.dropna(subset=['author', 'author_parent'])
edges = edges[edges['author'] != edges['author_parent']]

G = nx.from_pandas_edgelist(edges, 'author', 'author_parent', create_using=nx.DiGraph())

print(G.number_of_nodes(), "users")
print(G.number_of_edges(), "reply edges")

32846 users
131491 reply edges


# DOMINANT model

In [6]:
# prepare graph
data = from_networkx(G)

X = np.array([
    [G.nodes[n].get('lexical_diversity', 0),
     G.nodes[n].get('active_hours', 0),
     G.nodes[n].get('activity_entropy', 0),
     G.nodes[n].get('avg_text_similarity', 0)]
    for n in G.nodes
], dtype=np.float32)

data.x = torch.tensor(X)
data.edge_index = data.edge_index


In [7]:
# DOMINANT

class DOMINANT(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim=64, embedding_dim=32):
        super().__init__()
        self.gc1 = GCNConv(in_dim, hidden_dim)
        self.gc2 = GCNConv(hidden_dim, embedding_dim)

        # Decoders
        self.decoder_feat = torch.nn.Linear(embedding_dim, in_dim)
        self.decoder_adj = torch.nn.Linear(embedding_dim, embedding_dim)

    def forward(self, x, edge_index):
        z = F.relu(self.gc1(x, edge_index))
        z = self.gc2(z, edge_index)

        # Feature reconstruction
        x_hat = self.decoder_feat(z)

        # Adjacency reconstruction via dot product in embedding space
        adj_hat = torch.sigmoid(torch.mm(z, z.t()))

        return z, x_hat, adj_hat

In [8]:
adj_true = to_dense_adj(data.edge_index)[0]

model = DOMINANT(in_dim=data.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
lambda_adj = 0.5  # balance between feature and structure reconstruction

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    z, x_hat, adj_hat = model(data.x, data.edge_index)

    loss_feat = F.mse_loss(x_hat, data.x)
    loss_adj = F.mse_loss(adj_hat, adj_true)
    loss = loss_feat + lambda_adj * loss_adj

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss.item():.4f}")

Epoch 000, Total Loss: 0.1341
Epoch 020, Total Loss: 0.1252
Epoch 040, Total Loss: 0.1250
Epoch 060, Total Loss: 0.1250
Epoch 080, Total Loss: 0.1250
Epoch 100, Total Loss: 0.1250
Epoch 120, Total Loss: 0.1250
Epoch 140, Total Loss: 0.1250
Epoch 160, Total Loss: 0.1250
Epoch 180, Total Loss: 0.1250


In [9]:
model.eval()
with torch.no_grad():
    z, x_hat, adj_hat = model(data.x, data.edge_index)

# Feature reconstruction error
feat_error = torch.mean((data.x - x_hat) ** 2, dim=1)

# Structural reconstruction error (optional)
adj_error = torch.mean((adj_true - adj_hat) ** 2, dim=1)

# Combined anomaly score
anomaly_score = feat_error + lambda_adj * adj_error

# Add to graph or dataframe
import pandas as pd

scores_df = pd.DataFrame({
    'author': list(G.nodes),
    'anomaly_score': anomaly_score.cpu().numpy()
}).sort_values('anomaly_score', ascending=False)

print(scores_df.head(10))

                  author  anomaly_score
32845         Wobistdu99          0.125
0                dafones          0.125
1       ScreamForSilence          0.125
2      you_get_CMV_delta          0.125
3           SaggyBallsHD          0.125
4           Kakarot_faps          0.125
5           MadCervantes          0.125
6             OHMYCARROT          0.125
7          rickeyspanish          0.125
8            comrade-jim          0.125


In [10]:


top_users = scores_df.head(30)['author']
node_colors = [
    'red' if n in top_users.values else 'lightgray'
    for n in G.nodes
]

plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, node_color=node_colors, node_size=40, edge_color='gray', alpha=0.5)
plt.title("Red = Top 30 Likely Bots")
plt.show()

KeyboardInterrupt: 

<Figure size 1000x800 with 0 Axes>